In [1]:
w_python = !which python
if 'home' in w_python[0]:
    resources_prefix = ''
else:
    !pip install watermark filetype face_recognition

    from google.colab import drive
    drive.mount('/content/drive')

    resources_prefix = '/content/drive/MyDrive/Machine Learning/Notebooks/0602_Vision - Face Encoding/'

In [2]:
# !conda install -y watermark

In [3]:
%load_ext watermark
%watermark

Last updated: 2022-08-05T14:12:54.042276-05:00

Python implementation: CPython
Python version       : 3.9.13
IPython version      : 8.4.0

Compiler    : GCC 10.3.0
OS          : Linux
Release     : 5.10.102.1-microsoft-standard-WSL2
Machine     : x86_64
Processor   : x86_64
CPU cores   : 12
Architecture: 64bit



In [4]:
import cv2

import os

import filetype

import matplotlib.pyplot as plt

from ipywidgets import interact

import os

import face_recognition

from concurrent.futures import ThreadPoolExecutor, wait, ALL_COMPLETED

import pickle

from collections import Counter

## Resume

- El objetivo es realizar una identificación de los rostros de las personas.
- En la carpeta `photos` se encuentran las fotos con los rostros de las personas. Algunas fotos tienen más de un rostro.
- En la carpeta `targets` se encuentra una carpeta por cada persona que se desea encontrar. Las fotos se utilizarán para comparar cada rostro encontrado en las fotos de la carpeta `photos`. Las fotos incluidad en la carpeta `targets` sólo deben contener a la persona que se desea encontrar, no pueden haber fotos con más de 1 persona.

https://github.com/ageitgey/face_recognition

## Encoding targets

Loading encoded targets.

In [5]:
try:
    pkl_targets = f'{resources_prefix}_resources/target_encoded.pkl'
    
    with open(pkl_targets, 'rb') as file:
        target_dict = pickle.load(file)
except:
    target_dict = {}

target_dict

{'Courteney Cox': [array([-1.16092019e-01,  1.03129551e-01,  8.36261660e-02, -1.50132045e-01,
         -1.83266118e-01,  5.45580275e-02, -6.81164861e-02, -1.17516182e-01,
          2.66883522e-01, -9.79346260e-02,  1.81765631e-01, -9.41361941e-04,
         -2.67173111e-01,  8.16731229e-02, -3.99477594e-02,  1.77978113e-01,
         -2.51921743e-01, -1.30731344e-01, -2.42961869e-02, -1.58387527e-01,
         -6.72391057e-02,  4.76138033e-02,  2.98736803e-02,  1.00888580e-01,
         -1.64397836e-01, -2.91150272e-01, -1.55865327e-01, -4.29766774e-02,
          4.95762639e-02, -1.10525250e-01,  7.09766448e-02,  8.04018155e-02,
         -1.88522056e-01,  2.44915579e-02,  3.79643217e-02,  1.37328103e-01,
         -4.72211801e-02, -1.93583176e-01,  1.22214742e-01, -3.06461416e-02,
         -1.72326997e-01, -1.23831324e-01,  1.75987661e-01,  2.20831513e-01,
          2.65693694e-01, -7.99519867e-02,  5.74457720e-02, -3.03756949e-02,
          1.82638213e-01, -3.13914299e-01,  6.96756020e-02,

Se genera un diccionario con los nombres de las personas y la cantidad de fotos codificadas.

In [6]:
target_dict_count = {}

for target in target_dict:
    target_dict_count[target] = len(target_dict[target])

target_dict_count

{'Courteney Cox': 10,
 'David Schwimmer': 11,
 'Jennifer Aniston': 9,
 'Lisa Kudrow': 10,
 'Matt Leblanc': 12,
 'Matthew Perry': 12}

Verificando si se necesita codificar nuevamente.

In [7]:
targets_path = f'{resources_prefix}_resources/targets'

targets_dict_folder = {}

for target_name in os.listdir(targets_path):
    target_path = f'{targets_path}/{target_name}'

    if not os.path.isdir(target_path):
        continue

    count = 0

    for photo in os.listdir(target_path):
        image_path = f'{target_path}/{photo}'

        if not filetype.is_image(image_path):
            continue

        count += 1

    targets_dict_folder[target_name] = count

encode_targets = targets_dict_folder != target_dict_count

encode_targets

False

In [8]:
target_dict_count

{'Courteney Cox': 10,
 'David Schwimmer': 11,
 'Jennifer Aniston': 9,
 'Lisa Kudrow': 10,
 'Matt Leblanc': 12,
 'Matthew Perry': 12}

Encoding

In [9]:
def encoding_thread(target_path, target_name):
    for photo in os.listdir(target_path):
        image_path = f'{target_path}/{photo}'

        if not filetype.is_image(image_path):
            continue

        img = face_recognition.load_image_file(image_path)
        encodings = face_recognition.face_encodings(img)

        if len(encodings) != 1:
            print(f'\nInvalid {photo}, found {len(encodings)} faces instead of 1')
            continue

        target_dict[target_name].append(encodings[0])


def encoding():
    if not encode_targets:
        return

    with ThreadPoolExecutor(max_workers=1) as executor:
        targets_name = []
        futures = []

        for target_name in os.listdir(targets_path):

            target_path = f'{targets_path}/{target_name}'

            if not os.path.isdir(target_path):
                continue

            targets_name.append(target_name)
            target_dict[target_name] = []

            futures.append(executor.submit(encoding_thread, target_path, target_name))

        print(targets_name)
        n = len(futures)

        for i, future in enumerate(futures):
            wait(fs=[future], return_when=ALL_COMPLETED)
            print("\r Items {0} of {1}".format(i+1, n), end='')

        executor.shutdown(wait=True)


encoding()

In [10]:
target_dict

{'Courteney Cox': [array([-1.16092019e-01,  1.03129551e-01,  8.36261660e-02, -1.50132045e-01,
         -1.83266118e-01,  5.45580275e-02, -6.81164861e-02, -1.17516182e-01,
          2.66883522e-01, -9.79346260e-02,  1.81765631e-01, -9.41361941e-04,
         -2.67173111e-01,  8.16731229e-02, -3.99477594e-02,  1.77978113e-01,
         -2.51921743e-01, -1.30731344e-01, -2.42961869e-02, -1.58387527e-01,
         -6.72391057e-02,  4.76138033e-02,  2.98736803e-02,  1.00888580e-01,
         -1.64397836e-01, -2.91150272e-01, -1.55865327e-01, -4.29766774e-02,
          4.95762639e-02, -1.10525250e-01,  7.09766448e-02,  8.04018155e-02,
         -1.88522056e-01,  2.44915579e-02,  3.79643217e-02,  1.37328103e-01,
         -4.72211801e-02, -1.93583176e-01,  1.22214742e-01, -3.06461416e-02,
         -1.72326997e-01, -1.23831324e-01,  1.75987661e-01,  2.20831513e-01,
          2.65693694e-01, -7.99519867e-02,  5.74457720e-02, -3.03756949e-02,
          1.82638213e-01, -3.13914299e-01,  6.96756020e-02,

Saving encoded targets.

In [11]:
if encode_targets:
    with open(pkl_targets, 'wb') as file:
        pickle.dump(target_dict, file)

## Detecting faces from images

In [12]:
photos_path = f'{resources_prefix}_resources/photos'
photo_list = []

for photo in os.listdir(photos_path):
    image_path = f'{photos_path}/{photo}'

    if not filetype.is_image(image_path):
        continue

    photo_list.append(image_path)

In [13]:
@interact(photo_list=photo_list)
def detect(photo_list):
    plt.rcParams['figure.figsize'] = (15, 15)

    # Cargando la imagen
    # img = face_recognition.load_image_file(photo_list) # No reconoce cuando la imagen esta rotada

    img = cv2.imread(photo_list)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Ubicación de los rostros
    face_locations = face_recognition.face_locations(img)

    # Codificando los rostros encontrados
    face_encodings = face_recognition.face_encodings(img, face_locations)

    # Por cada rostro encontrado
    for pos, encode in enumerate(face_encodings):
        # Se colocan los nombres que coincidan
        face_names = []

        # Por cada target almacenado
        for target_name in target_dict:
            # Se compara el rostro con el los rostro de los target almacenados y se obtiene una lista de bool
            result = face_recognition.compare_faces(known_face_encodings=target_dict[target_name], 
                                                    face_encoding_to_check=encode, tolerance=0.6)

            # Se pasa a un Counter para verificar si existen más True que False
            counter = Counter(result)

            if counter[True] > counter[False]:
                face_names.append(target_name)

        # Se concatenan los nombres que coincidan
        names = ', '.join(face_names)

        # Al ser face_encodings y face_locations listas paralelas, se puede usar el mismo pos
        top, right, bottom, left = face_locations[pos]
        
        org = (left, top - 10)
        font = cv2.FONT_HERSHEY_COMPLEX
        color = (0, 0, 200)
        
        cv2.putText(img=img, text=names, org=org, fontFace=font, fontScale=0.4, color=color, thickness=1)
        
        cv2.rectangle(img=img, pt1=(left, top), pt2=(right, bottom), color=color, thickness=1)    
    
    plt.imshow(img)
    plt.show()

interactive(children=(Dropdown(description='photo_list', options=('_resources/photos/064206_1157388.jpg', '_re…

## Detecting faces from camera

Listando las cámaras disponibles.

In [14]:
devs = os.listdir('/dev')

vid_indices = [int(dev[-1]) for dev in devs if dev.startswith('video')]
vid_indices = sorted(vid_indices)
vid_indices

[]

Load camera.

In [15]:
cap = cv2.VideoCapture(2)

[ WARN:0] global ../modules/videoio/src/cap_v4l.cpp (893) open VIDEOIO(V4L2:/dev/video2): can't open camera by index


Read camera.

In [16]:
# while True:
#     ret, frame = cap.read()

#     # Detect Faces
#     face_locations, face_names = sfr.detect_known_faces(frame)

#     for face_loc, name in zip(face_locations, face_names):
#         y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]

#         cv2.putText(frame, name, (x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 200), 2)
#         cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 200), 4)

#     cv2.imshow("Frame", frame)

#     key = cv2.waitKey(1)
#     if key == 27:
#         break

In [17]:
# # Analizar el sguiente codigo

# import cv2
# from simple_facerec import SimpleFacerec

# # Encode faces from a folder
# sfr = SimpleFacerec()
# sfr.load_encoding_images("images/")

# # Load Camera
# cap = cv2.VideoCapture(2)


# while True:
#     ret, frame = cap.read()

#     # Detect Faces
#     face_locations, face_names = sfr.detect_known_faces(frame)
#     for face_loc, name in zip(face_locations, face_names):
#         y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]

#         cv2.putText(frame, name,(x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 200), 2)
#         cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 200), 4)

#     cv2.imshow("Frame", frame)

#     key = cv2.waitKey(1)
#     if key == 27:
#         break

# cap.release()
# cv2.destroyAllWindows()

# def detect_known_faces(self, frame):
#         small_frame = cv2.resize(frame, (0, 0), fx=self.frame_resizing, fy=self.frame_resizing)
#         # Find all the faces and face encodings in the current frame of video
#         # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
#         rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
#         face_locations = face_recognition.face_locations(rgb_small_frame)
#         face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

#         face_names = []
#         for face_encoding in face_encodings:
#             # See if the face is a match for the known face(s)
#             matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
#             name = "Unknown"

#             # # If a match was found in known_face_encodings, just use the first one.
#             # if True in matches:
#             #     first_match_index = matches.index(True)
#             #     name = known_face_names[first_match_index]

#             # Or instead, use the known face with the smallest distance to the new face
#             face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
#             best_match_index = np.argmin(face_distances)
#             if matches[best_match_index]:
#                 name = self.known_face_names[best_match_index]
#             face_names.append(name)

#         # Convert to numpy array to adjust coordinates with frame resizing quickly
#         face_locations = np.array(face_locations)
#         face_locations = face_locations / self.frame_resizing
#         return face_locations.astype(int), face_names